# Линейная алгебра. Тензор (10 баллов)

В этой домашней работе мы продолжим работу над нашей библиотекой машинного обучения, дизайн которой мы заимствуем у библиотеки **PyTorch**. Необходимо будет дописать функционал тензора, относящийся к многомерным тензорам: добавить *решейп*, транспонирование, расширить индексирование и реализовать матричное умножение, ну и по мелочи еще всяких операций.

## Что уже сделано

Частично функционал класса `Tensor` уже реализован автором, частично дописан в первой части. Что есть в авторском коде (не поленитесь изучить его, там немного):

1. Конструктор, принимающий число, реализующее интерфейс `number.Number`. Он создает тензор нулевой размерности, хранящий это число в поле `_data`;
2. Конструктор, принимающий инстанс (объект класса) `Tensor`. Из него создается `shadow copy` (теневая копия) переданного тензора путем простого копирования ссылок на уже существующий буфер переданного тензора;
3. Конструктор, принимающий объект, реализующий интерфейс `Iterable`. Этот итератор должнен обходить только числа с интерфейсом `numbers.Number`;
4. Магический метод `__repr__`, который умеет красиво выводить ноль-одно-двумерные тензоры;
5. Метод `allclose`, который сравнивает два тензора по значению;
6. Функция `randn`, которая возвращает объект `Tensor` заданной формы, заполняя его координаты случайными значениями из нормального распределения $\mathcal{N}(0, 1)$;

Что должно быть реализовано в прошлой части:

7. Свойство `shape`, которое возвращает кортеж с размеростями тензора. В нашем случае для вектора это кортеж из одного числа -- длины вектора;
8. Функция `vector_norm`, которая вычисляет метрику Минковского для векторов;
9. Методы `add`, `mul` и `sub` реализующие сложение тензора с числом/нульмерным тензором/другим тензором той же размерности;
10. Методы `gt` *(greater than)* и `lt` *(less than)*, по-русски больше или меньше, которые реализуют покоординатное сравнение с числом/нульмерным тензором/другим тензором;
11. Метод `neg`, меняющий знак у каждого элемента тензора;
12. Магические методы `__add__`, `__mul__`, `__sub__`, перегружающие операторы `+`, `*`, `-` и действующие так же, как и методы `add`, `mul`, `sub`, например `tensor + other`, `tensor * other` и `tensor - other`;
13. Магические методы `__radd__`, `__rmul__`, `__rsub__`, которые отличаются правоассоциативностью, в отличие от пункта выше. Выглядит они так же, но тензор стоит после оператора, например `other + tensor`, `other * tensor` и `other - tensor`.
14. Магические методы `__gt__` и `__lt__`, перегружающие операторы `>` и `<` и действующие так же, как и методы `gt` и `lt`, например `tensor > other` или `tensor < other`;
15. Магический метод `__neg__`, перегружающий унарный `-` и и действующий как метод `neg`;
16. Магический метод `__len__`, возвращающий количество элементов в тензоре по первой размерности;
17. Магический метод `__eq__`, перегружающий оператор `==` и вызывающий `allclose`;
18. Магический метод `__iter__`, который реализует итератор по элементам тензора учитывая только первую размерность;
19. Магический метод `__getitem__`, который возвращает `i-ю` координату вектора;
20. Функции `add`, `mul` и `sub`, повторящие функционал соответствующих методов тензора;
21. Функции `zeros`, `ones` и `tensor`, создающие тензор из нулей/единиц/списка чисел.

Это все, что необходимо для продолжения работы. Чтобы проверить, что все работает верно, запускайте тесты следующей командой (*и в ячейке ниже*):

In [ ]:
!pytest twinkle/tests/test_vector.py -v

## Что будем делать

В следующей части работы предстоит написать тренировочный цикл, реализующий градиентный спуск по квадратичной функции потерь. Но это будет потом, а пока предстоит расширить функциональность тензора для работы с матричными операциями, которые нам понадобятся. Если тезисно, то реализовать нужно:

1. Метод `reshape`, изменяющий форму тензора и не трогающий его внутренний буфер;
2. Метод `flatten`, превращающий многомерный тензор в плоский одномерный тензор-вектор;
3. Метод `__getitem__`, возвращающий i-й элементов тензора по первой размерности;
4. Метод `mm` и магический метод `__matmul__`, перегружаеющий оператор `@`. Этот метод реализует матричное умножение;
5. Метод `argmax`, возвращающий *индекс* максимального элемента в тензоре;
6. Свойство `T`, транспонирующее матрицу (как текущий тензор);
7. Функцию `eye`, создающую единичную матрицу;
8. Функцию `to_categorical`, конвертирующую вектор значений в *one-hot encoded* представление.

Для всего этого написаны тесты, которые помогут вам проверить правильность вашего кода. Что ж, теперь начнем по порядку :)

# Многомерные тензоры (2 балла)

Сейчас наш тензор имеет свойство `shape`, которое представляет собой кортеж из одного элемента -- длины одномерного тензора-вектора. Расширим тензор, добавив методы `reshape` и `flatten`, а также добавив поддержку всех математических операций для многомерного случая.

## Метод `reshape`, смена формы (0.5 балла)

В предыдущей части работы были реализованы одномерные тензоры-вектор. Если вкратце -- внутри тензора есть плоский массив с элементами, отдельно лежит поле `_shape` хранящее кортеж, в первом элементе которого лежит количество элементов в тензоре. Самый простой способ реализовать многомерные тензоры -- просто подменять поле `_shape`, в котором держать какую-то многомерную форму тензора. Это и предлагается сделать в методе `reshape`. Он возвращает новый тензор с заданным шейпом (из аргумента метода `reshape`). Следует помнить, что нужно соблюдать инвариант -- количество элементов в тензоре до и после решейпа должно быть одинаковым.

*Подсказка: быстро скопировать тензор можно передав его в конструктор класса `Tensor`, например как в куске кода ниже:*

```
tensor = Tensor(range(6))
copy = Tensor(tensor)
```

Для проверки правильности своего решения запускайте тесты следующей командой (*и в ячейке ниже*):

`pytest . -k test_reshape -v`

In [ ]:
!pytest . -k test_reshape -v

## Метод `flatten` (0.5 балл)

Обратной операцией к решейпу является `flatten`. Эта операция превращает многомерный тензор в одномерный тензор-вектор. Реализуйте метод `flatten`, возвращая новый тензор и записывая в его шейп кортеж с единственным элементом -- количеством элементов в тензоре.

Для проверки правильности своего решения запускайте тесты следующей командой (*и в ячейке ниже*):

`pytest . -k test_flatten -v`

In [ ]:
!pytest . -k test_flatten -v

## Математические операции (1 балл)

Ранее в тензоре были реализованы методы математических операций: `add`, `mul`, `sub`, `neg`, `gt` и `lt`. Они прекрасно работают с векторами, но этого явно недостаточно для удобной работы. Расширьте функционал этих методов так, чтобы они корректно работали проводили операции с числами/скалярными тензорами/многомерного тензорами такой же формы. Для этого нужно перешейпить тензоры с результатами операций, взяв форму из исходного тензора.

*Подсказка: в авторском решении все операции реализованы через приватный метод `_binary_op`. Всего дописано четыре вызова `reshape`, хотя при желании можно было бы обойтись и одним.*

Для проверки правильности своего решения запускайте тесты следующей командой (*и в ячейке ниже*):

`pytest . -k test_tensor_math -v`

In [ ]:
!pytest . -k test_tensor_math -v

## Примеры использования

Давайте создадим несколько тензоров и посмотрим, как пользоваться реализованным функционалом. Для этого из одного и того же тензора-вектора с шестью элементами создадим вектор-строку, матрицу 2х3, матрицу 3х2 и вектор столбец. Для каждого случая применим какую-нибудь математическую операцию.

In [ ]:
import twinkle as tw

print(tw.tensor(range(6)).reshape(1, 6) + 1, '\n')
print(tw.tensor(range(6)).reshape(2, 3) * 2, '\n')
print(tw.tensor(range(6)).reshape(3, 2) - tw.tensor(42), '\n')
print(tw.tensor(range(6)).reshape(6, 1) > 2)

# Матричное умножение (2 балла)

Теперь вы знаете, что искуственный интеллект это просто перемножение матриц. Матрицы можно сделать с помощью решейпа, осталось только научиться эти матрицы перемножать. Для этого у тензора есть метод `mm` и магический метод `__matmul__`, перегружающий оператор `@`.

## Метод `mm`, matrix multipy (1.5 балла)

Реализуйте матричное умножение в методе `mm` за $O(n^3)$ от размеров перемножаемых матриц. Этот метод принимает тензор-матрицу и проверяет корректность размеров перемножаемых тензоров. Перемножение работает напрямую с внутренними буферами двух тензоров, а значит тут надо просто аккуратно поработать с индексами.

Основа метода -- тройной цикл: по строкам первой матрицы, по столбцам второй матрицы и по элементам суммы скалярного произведения строки на столбец. Вам нужно вычислить индекс позиции для скалярного произведения в новом тензоре и индексы перемножаемых элементов в двух данных матрицах, `self` и `other`. Места куда нужно записать индексы отмечены многоточием `...`

Для проверки правильности своего решения запускайте тесты следующей командой (*и в ячейке ниже*):

`pytest . -k test_mm -v`

In [ ]:
!pytest . -k test_mm -v

## Магический метод `__matmul__`, перегрузка оператора `@` (0.5 балла)

Кому-то может показаться неудобным пользоваться методом матричного умножения там, где хочется использовать бинарный оператор наподобие `+` или `-`. Для этих целей в питоне зарезервирован оператор матричного умножения `@`. Для его перегрузки необходимо реализовать метод `__matmul__`, что и предлагается сделать.

Для проверки правильности своего решения запускайте тесты следующей командой (*и в ячейке ниже*):

`pytest . -k test_magic_mm -v`

In [ ]:
!pytest . -k test_magic_mm -v

## Примеры использования

Тут нет ничего хитрого, нужно создать матрицы (проверьте, что это точно матрицы, в шейпе должно быть два элемента!), после чего перемножить их через `mm` или `@`. Обратите внимание, что внутри метода есть проверка: вторая размерность первой матрицы должна совпадать с первой размерностью второй матрицы.

In [ ]:
import twinkle as tw

a = tw.tensor(range(6)).reshape(3, 2)
b = tw.tensor(range(6)).reshape(2, 3)
print(a.mm(b), '\n')

a = tw.tensor(range(6)).reshape(2, 3)
b = tw.tensor(range(6)).reshape(3, 2)
print(a @ b)

# Обновленный индексер и итератор (2 балла)

Часто в машинном обучении необходимо перебирать объекты и выбирать их по индексу. Рассмотрим пример: нужно написать классификатор изображений с рукописными цифрами, каждое изображение имеет размер 28 на 28 пикселей и три канала, `RGB` (красный, зеленый и синий). Тогда одна картинка будет представлена тензором размером `(28, 28, 3)`. Если таких картинок в обучающей выборке 1000 штук, тогда весь датасет можно представить тензором `(1000, 28, 28, 3)`. Это очень удобно для работы, но нужно уметь выбирать нужную картинку из тысячи по индексу и уметь обходить все картинки по порядку. Кроме того, иногда полезно выбирать сразу несколько объектов, передавая в квадратные скобки тензор-вектор с индексами нужных элементов.

## Магический метод `__getitem__`, индексер (1.5 балла)

Теперь, если в `__getitem__` передается целое число, то для многомерного тензора метод должен возвращать тензор размерности на единицу меньше, например для тензора `(n, x1, x2, ...)` это будет тензор `(x1, x2, ...)`. Из примера про картинки это тензор размером `(28, 28, 3)`. Если же передать туда тензор с индексами объектов (там должны быть целые числа!), то должен вернуться тензор, в первой размерности которого будет столько значений, сколько было передано индексов. Например при передаче индексов в виде `Tensor([0, 1, 2, 3])` нужно выбрать четыре картинки и вернуть тензор размером `(4, 28, 28, 3)`. Реализуйте эту функциональность, расширив метод `__getitem__`.

Для проверки правильности своего решения запускайте тесты следующей командой (*и в ячейке ниже*):

`pytest . -k test_getitem -v`

In [ ]:
!pytest . -k test_getitem -v

## Магический метод `__iter__`, итератор (0.5 балла)

Порой объекты удобно перебирать по одному, например в реализации *стохастического градиентного спуска*. Для этого можно либо воспользоваться циклом по индексам и предыдущим методом, либо реализовать *итератор* по первой размерности тензора. Проверьте, что уже написанная реализация метода `__iter__` корректно работает с новым индексером.

*Подсказка: в этой секции не нужно писать нового кода, если вы его пишете, то что-то пошло не так.*

Для проверки правильности своего решения запускайте тесты следующей командой (*и в ячейке ниже*):

`pytest . -k test_iter -v`

In [ ]:
!pytest . -k test_iter -v

## Примеры использования

Тут все просто. Представим, что есть пять объектов по два признака каждый.

Создадим тензор из десяти элементов, зарешейпим его в матрицу с пятью строками и выберем пару этих строк. После этого в цикле `for in` обойдем все строки этой матрицы.

In [ ]:
import twinkle as tw

foo = tw.tensor(range(10)).reshape(5, 2)
print('test __getitem__')
print(foo[1])
print(foo[3])

print('\ntest __iter__')
for item in foo:
    print(item)

# Разное, но полезное (4 балла)

Для удобства работы в процессе обучения необходимо будет находить индекс максимального элемента, уметь быстро транспонировать матрицы, а также уметь `one-hot` кодировать признаки. Для этого у тензора необходимо реализовать метод `argmax` и свойство `T`, и еще две отдельные функции `eye` и `to_categorical`. Начнем по порядку.

## Метод `argmax` (1 балл)

Данный метод возвращает *индекс* максимального элемента тензора. Можно считать, что максимум будет вычисляться только для плоских тензоров-векторов, матриц-строк или матриц столбцов. Для такой реализации можно не учитывать шейп тензора, а сразу искать индекс максимального элемента во внутреннем буфере. Реализуйте эту функциональность в методе `argmax`.

Для проверки правильности своего решения запускайте тесты следующей командой (*и в ячейке ниже*):

`pytest . -k test_argmax -v`

In [ ]:
!pytest . -k test_argmax -v

## Свойство `T`, transpose (1 балл)

При обучении линейных слоев часто нужно использовать операцию транспонирования. Транспонирование двумерного тензора-матрицы это смена строк и столбцов местами. В умных библиотеках математики, таких как `numpy`, `torch` и `tensorflow` операции транспонирования выполняются за $O(1)$ и бесплатны. Нам же достаточно реализовать их наивно, создав новый тензор и правильно заполнив его элементы. Реализуйте свойство `T` с данным функционалом.

Для проверки правильности своего решения запускайте тесты следующей командой (*и в ячейке ниже*):

`pytest . -k test_tensor_t -v`

In [ ]:
!pytest . -k test_tensor_t -v

## Магический метод `__len__`

Раньше метод `__len__` возвращал количество элементов в одномерном тензоре-векторе, теперь он должен возвращать количество элементов по первой размерности тензора. Проверьте, что это так, скорее всего оно будет работать. Если нет, то поправьте реализацию метода.

Для проверки правильности своего решения запускайте тесты следующей командой (*и в ячейке ниже*):

`pytest . -k test_len -v`

In [ ]:
!pytest . -k test_len -v

## Функция `eye`, единичная матрица (1 балл)

Реализуйте функцию, которая возвращает квадратную единичную матрицу с заданной стороной `n`. Эта функция понадобится в следующем пункте.

Для проверки правильности своего решения запускайте тесты следующей командой (*и в ячейке ниже*):

`pytest . -k test_eye -v`

In [ ]:
!pytest . -k test_eye -v

## Функция `to_categorical`, one-hot encoding (1 балл)

Реализуйте функцию `to_categorical`, которая реализует `one-hot` кодирование для категориальных признаков. Функция принимает два аргумента:

* `y`, тензор-вектор с индексами классов;
* `num_classes`, количество классов.

Функция работает по следующему алгоритму: сначала создается единичная матрица размером `num_classes`, а потом из нее набираются нужные строки, соответствующие индексам классов в `y`. Например у нас есть всего три класса и четыре объекта, один класса 0, два класса 1 и еще один класса 2. Тогда получается, что вектор классов `[0, 1, 1, 2]` преобразуется в `one-hot` матрицу таким образом:

```
[1, 0, 0]
[0, 1, 0]
[0, 1, 0]
[0, 0, 1]
```

*Подсказка: при реализации данной функции воспользуйтесь уже готовыми функцией `eye` и индексером `__getitem__`.*

Для проверки правильности своего решения запускайте тесты следующей командой (*и в ячейке ниже*):

`pytest . -k test_to_categorical -v`

In [ ]:
!pytest . -k test_to_categorical -v

## Проверка всего

Для успешной реализации следующей части работы нужно проверить, что весь уже написанный код работает хорошо, а значит должны пройти **все** тесты. Если что-то не проходит -- вернитесь к нужной секции и проверьте все еще раз.

Запустите тесты следующей командой (*и в ячейке ниже*):

`pytest . -v`

In [ ]:
!pytest . -v

# Резюме

В этой части мы вплотную подобрались к написанию алгоритмов машинного обучения, создав прочный фундамент для работы с объектами линейной алгебры, такими как векторы и матрицы.

Продолжение следует!